# Migration to and from the United Kingdom by area of destination or origin within the UK by citizenship

In [1]:
%load_ext pep8magic

In [2]:
%%pep8
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/'
                  'populationandmigration/internationalmigration/datasets/'
                  'ipsareaofdestinationororiginwithintheukbycitizenship')
scraper

In [3]:
%%pep8
tabs = scraper.distributions[0].as_databaker()

Each tab is of the same form, with "software readable codes":

> The datasheets can be imported directly into suitable software. When importing the datasheets into other software import only rows 8 to 53, starting at column E.

Actually, the admin geo code is also useful, so we'll start at column D

In [4]:
%%pep8
def citizenship_code(s):
    code = pathify(s)
    assert code.startswith('cit-'), code
    code = code[4:]
    assert code.endswith('-est'), code
    code = code[:-4]
    return code.replace('-/-', '-')

tidied_sheets = []

# If we're not using the tabs, get them out of memory
tabs = [x for x in tabs if x.name.startswith("Data")]

for tab in tabs:

    # ---------
    # Selection

    # We're gonna anchor on a hidden (as in white-text) cell at the
    # intersection of the software readable row and column labels
    anchor = tab.filter("Row Label").assert_one()

    # srrl/srcl is "Software-reabable row/column labels"
    srrl = anchor.fill(DOWN).is_not_blank()
    srcl = anchor.fill(RIGHT).is_not_blank()

    # sheets B, C and D repeat 'All citizenships', 'British' and 'Stateless'
    # from sheet so we remove software readable columns with those headings
    if not tab.name.endswith('A'):
        srcl = srcl - srcl.regex(r'CIT (All|British|Stateless)')

    # Create different selection for the columns containing EST and CI from
    # the software readable column labels
    estCols = srcl.filter(lambda x: x.value.strip().endswith("EST"))
    ciCols = srcl.filter(lambda x: x.value.strip().endswith("CI"))
    msg = "The selection of CI and EST columns should be of equal length"
    assert len(estCols) == len(ciCols), msg

    # Use a waffle to get the observations and ci selections
    observations = srrl.waffle(estCols)
    observations_ci = srrl.waffle(ciCols)

    # Get the year from the last 4 characters in cell A2
    year = tab.excel_ref('A2').value[-4:]
    assert int(year), "No valid integer as year ending cell A2"

    # Get the geography from every cell below and including D9
    # then regex to exact match 1 character + 8 digits
    geography = tab.excel_ref("D9").expand(DOWN).regex(r'^[A-Z]\d{8}$')

    # ---------
    # Dimension

    dimensions = [
        HDimConst('Year', year),
        HDim(geography, 'Area of Destination or Origin', DIRECTLY, LEFT),
        HDim(srrl, 'Migration Flow', DIRECTLY, LEFT),
        HDim(estCols, 'IPS Citizenship', DIRECTLY, ABOVE),
        HDim(observations_ci, 'CI', DIRECTLY, RIGHT),
        HDimConst('Measure Type', 'Count'),
        HDimConst('Unit', 'people-thousands')
    ]

    # -------
    # Process

    cs_est = ConversionSegment(observations, dimensions)
    #savepreviewhtml(cs_est)
    tidy_sheet = cs_est.topandas()

    # ------------
    # Post Process

    # Tidy up some values
    tidy_sheet['IPS Citizenship'].apply(citizenship_code)
    tidy_sheet['Migration Flow'].map(
        lambda x: x.split(", ")[0].lower())

    # Drop unused data marker column
    # Todo: data markers
    tidy_sheet = tidy_sheet[pd.isna(tidy_sheet['DATAMARKER'])].copy()
    tidy_sheet.drop(columns=['DATAMARKER'], inplace=True)

    # Ordering and label names
    tidy_sheet.rename(columns={'OBS': 'Value'}, inplace=True)
    tidy_sheet = tidy_sheet[['Year', 'Area of Destination or Origin',
                             'Migration Flow', 'IPS Citizenship', 'CI',
                             'Value', 'Measure Type', 'Unit']]
    # Done
    tidied_sheets.append(tidy_sheet)


stdin:68:5: E265 block comment should start with '# '
    #savepreviewhtml(cs_est)
    ^


TypeError: 'int' object is not iterable

In [24]:
%%pep8
import pandas as pd

tidy = pd.concat(tidied_sheets)

out = Path('out')
out.mkdir(exist_ok=True, parents=True)
tidy.to_csv(out / 'observations.csv', index=False)

tidy[:5]


This code is PEP8-compliant!


In [25]:
%%pep8
scraper.dataset.family = 'migration'
with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())

This code is PEP8-compliant!
